In [1]:
import os 

In [2]:
%pwd

'/Users/bivasbisht/Deep-Learning_project/kidney_classification_deep-learnig/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/bivasbisht/Deep-Learning_project/kidney_classification_deep-learnig'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/bivas16/kidney_classification_deep-learnig.mlflow "
os.environ["MLFLOW_TRACKING_USERNAME"]="bivas16"
os.environ["MLFLOW_TRACKING_PASSWORD"]="8a1d9aba0bf5c522f963952e67e0f6bb42531b28"

In [6]:
import tensorflow as tf 

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

2023-10-18 20:08:04.206377: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-18 20:08:04.206404: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-10-18 20:08:04.206411: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-10-18 20:08:04.206449: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-18 20:08:04.206472: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_injestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/bivas16/kidney_classification_deep-learnig.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2023-10-18 20:08:05,100: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-18 20:08:05,102: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-18 20:08:05,103: INFO: common: created directory at: artifacts]
Found 2207 images belonging to 2 classes.


2023-10-18 20:08:05.698605: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


138/138 [==============================] - 66s 474ms/step - loss: 2.6623 - accuracy: 0.6932
[2023-10-18 20:09:11,359: INFO: common: json file saved at: scores.json]


2023/10/18 20:09:14 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/jp/ny5f6r_j36v_hmzh6cgr5_gw0000gn/T/tmpojxmltme/model/data/model/assets
[2023-10-18 20:09:15,313: INFO: builder_impl: Assets written to: /var/folders/jp/ny5f6r_j36v_hmzh6cgr5_gw0000gn/T/tmpojxmltme/model/data/model/assets]


/Users/bivasbisht/miniforge3/envs/kidney/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2023/10/18 20:09:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


In [14]:
print(tf.__version__)

2.13.0
